# TP 4: Clustering

## Quick Recap: Clustering

Clustering is an **unsupervised learning** technique used to group similar data points based on a distance or similarity measure without using labels for training.

Main goals:
- Group data so that points within the same cluster are similar
- Ensure different clusters are well separated

Example methods: **K-Means**, **DBSCAN**, **Hierarchical Clustering**

## 📝 Exercise 1: K-Means Clustering

In this exercise, you will apply **K-Means Clustering** to the credit card dataset. 

First, let's import the necessary libraries for data manipulation, visualization and machine learning. We also set random seeds to ensure our results are reproducible.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, roc_curve

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)

Load the `creditcard.csv` file into a pandas DataFrame to inspect the first few rows.

**Dataset Overview:**
* **Goal:** Identify fraudulent credit card transactions.
* **Features:** Anonymized input variables transformed via Principal Component Analysis (PCA), meaning they lack specific semantic descriptions.
* **Ground Truth:** The dataset includes labels distinguishing normal transactions from anomalies.

In [2]:
# Load the data
df = pd.read_csv('creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,'0'
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,'0'
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,'0'
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,'0'
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,'0'


**Class Distribution:** 

Inspect the frequency of each class label to check the dataset balance. Observe that the anomalies (fraudulent transactions) represent a significant minority compared to normal transactions.

In [3]:
df['Class'].value_counts()

Class
'0'    284315
'1'       492
Name: count, dtype: int64

**Feature Selection:**

Remove the `Time` column from the dataframe, since the sequential timestamp does not provide meaningful information for the K-Means algorithm.

In [4]:
df = df.drop('Time', axis=1)
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,'0'
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,'0'
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,'0'
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,'0'
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,'0'


**Separating Features and Targets:**

Isolate the input features from the ground truth labels for evaluation.

* `X`: Defines the feature matrix by dropping the label column `Class`.
* `y`: Extract the target labels, ensuring they are formatted as integers (1 for anomaly, 0 for normal) to handle potential string formatting in the raw data.

In [10]:
X = df.drop('Class', axis=1)

y = [1 if c=="'1'" else 0 for c in df['Class']]

X.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99


**Feature Scaling:**

Distance-based algorithms (like K-Means) are highly sensitive to the magnitude of variables. Without scaling, a feature with a large range (e.g., `Amount`) will dominate the distance calculations over features with smaller ranges.

Apply `StandardScaler` to standardize the data, ensuring each feature contributes equally to the distance metric by centering the mean at 0 and scaling the variance to 1.

In [11]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Before we attempt to detect anomalies or optimize the number of clusters, let's perform a "test run" of K-Means to understand the output.

We will arbitrarily choose K=3 clusters. The algorithm will partition the dataset into three distinct groups based on feature similarity.

**Note:** Why do we try **k = 3** here?
Even though the fraud labels are binary (fraud vs. normal), the data itself may contain more than two *natural* groups.
For example:
- Normal transactions may form different behavioral clusters
- Fraudulent transactions may not all look similar

Clustering aims to discover structure in the data not just reproduce labels.

So using **k = 3** helps us see if there are multiple normal subgroups and possibly a more distinct anomaly group.

👉 This exercise is mainly to help you understand the concept of choosing the number of clusters based on the data structure.  
Later on, we will also work with **k = 2** to align directly with the binary fraud labels and compare the results.



#### Question:

1. Initialize the K-Means model with n_clusters=3.

2. Fit the model using `X_scaled` and predict the clusters.

3. Print the first 100 elements of the clusters array to inspect the assignments for the first 100 transactions.

In [ ]:
## TODO: 

**Inertia:**

In K-Means, Inertia measures how well the clusters are formed. It is calculated as the sum of squared distances between each data point and its closest centroid.

- **Lower Inertia** generally means the clusters are dense and points are close to their centroids (good).

- **Higher Inertia** means the points are spread out (bad).

#### Question:

4. Access and print the inertia of your fitted model using the `.inertia_` attribute.

In [ ]:
## TODO:

**Random Initializations:**

K-Means is sensitive to the starting positions of the centroids. A "bad" random start can lead to a suboptimal solution (higher inertia). To solve this, the algorithm runs multiple times (controlled by the `n_init` parameter) and keeps the best result.

#### Question:

5. Run the algorithm with increasing numbers of initializations to see when the performance stabilizes. To keep the execution fast, perform this analysis on a smaller subset of the data.

    - Create a subset of 10,000 samples.

    - Iterate `n_init` from 10 to 200 (in steps of 10).

    - Store and plot the inertia for each run.

6. You should see the inertia dropping and then flattening out as the number of initializations increases. Based on the plot, what is a "good" value for `n_init`? Why did you choose this value? Briefly explain your reasoning. 

7. Set your chosen value in the code variable below for next questions. 

In [ ]:
## TODO:

**Silhouette Score:**

While Inertia measures how tight the clusters are, it doesn't tell us how well-separated they are from each other. The **Silhouette Score** is a better metric for this. It measures how similar a point is to its own cluster (cohesion) compared to other clusters (separation).

  - **Range:** -1 to +1.
  - **High score (close to +1):** The point is well matched to its own cluster and far from neighboring clusters.
  - **Low score (close to 0):** The point is on or very close to the decision boundary between two neighboring clusters.

**Important Note:** Calculating the Silhouette Score requires computing distances between every pair of points, which is computationally expensive ($O(N^2)$). **Do not run this on the full dataset.** Use the subset `X_sub` you created earlier.

#### Question:

6. Fit the model on the subset X_sub.

7. Calculate the `silhouette_score` using the data and the predicted labels.

In [ ]:
## TODO:

#### Question:

You will now proceed with the complete anomaly detection pipeline using the optimal hyperparameters we found ($K=2$ and `n_init` from previous step). 

8. Separate the data into training and test set. 

9. Run K-Means on the training set.

10. Find the anomaly scores of the samples.

11. Plot the precision-recall curve and the ROC curve.

12. Choose a threshold (a cut-off value for the anomaly score based on the curves (e.g., aiming for high recall)).

13. Use that threshold on the test set.

14. Check precision and recall again.

In [ ]:
## TODO:

## 📝 Exercise 2: Choosing and Justifying a Clustering Model

In this exercise, you will use the datasets `dataset1`, `dataset2` and `dataset3`. Different datasets have different shapes and characteristics. You will analyze the data first and then choose the most appropriate clustering method based on what you observe.

### Question:

For each dataset:

1. Load the dataset and visualize the data distribution. Plot the points to understand how clusters look.

2. Propose a clustering algorithm suitable for this dataset. Consider the type of structure you see.
    - Are clusters spherical or elongated?
    - Do clusters have irregular shapes?
    - Are there outliers or noise?

3. Justify your choice. Explain why the algorithm you selected is suitable for that dataset by referring to the shape of clusters, presence of outliers and/or density structure.

In [ ]:
## TODO: dataset1

In [ ]:
## TODO: dataset2

In [ ]:
## TODO: dataset3

## 📝 Exercise 3: Hierarchical Clustering

In this exercise, you will apply **Hierarchical Clustering** to the credit card dataset. 

Unlike K-Means, this method builds a hierarchy of clusters. We can visualize this hierarchy using a **dendrogram**, which helps us decide the optimal number of clusters without guessing beforehand.

**Reminder:**

Hierarchical clustering is computationally expensive ($O(N^2)$ or $O(N^3)$). Running it on the full 280,000+ transaction dataset will likely crash the kernel and the dendrogram would become unreadable. Work on the smaller dataset `creditcard_sample.csv` for this exercise.

#### Question:

1. Compute hierarchical clustering using the following linkage strategies. Try each one and compare how the clusters are formed:

    - complete linkage
    - average linkage
    - single linkage

In [ ]:
## TODO:

#### Question: 

2. Plot the **dendrogram** for the three linkage cases.

    - Observe how clusters merge at different heights.
    - Check how linkage choice affects the visual structure.

In [ ]:
## TODO:

#### Question:

3. Choose a number of clusters and visualize the result.

📌 *How to choose the number of clusters?*  

- Look for the largest vertical jump in the dendrogram without any horizontal line crossing it.  
- This point indicates where clusters are significantly merging → suggesting natural groups exist below that height.

Apply this rule to each dendrogram and justify your choice. Visualize the resulting clusters (in 2D).

**Reminder:** Hierarchical clustering does not require you to set the number of clusters beforehand, the dendrogram helps you decide based on data structure.

In [ ]:
## TODO: